<h2 align='center'> Tugas Minggu 9 Deep Learning </h2>
<br>
<hr>
<br>
Nama: Muh. Yusuf Syam
<br>
NIM: H071191044
<br>
Soal: Modern RNN <a href='https://d2l.ai/chapter_attention-mechanisms/index.html'> D2L.AI Chapter 10. Attention Mechanism - Dive into Deep Learning<a/>

Attention adalah mekanisme deep learning yang memutuskan bagian mana yang perlu difokuskan dan bagian mana yang harus diabaikan. Seperti neural network, attention juga meniru cara kerja otak manusia yang mengabaikan sebagian besar informasi yang diterima, dan hanya memperhatikan proporsi kecil dari informasi yang dianggap penting, misalnya saat seseorang membaca buku, ia hanya fokus pada kata atau kalimat yang sedang ia baca, bukan pada keseluruhan buku. 

Salah satu keuntungan dari menggunakan mekanisme attention yaitu proses komputasi jauh lebih murah.

### Syarat Attention

### Attention Poooling

### Fungsi Attention Scoring

### Bahdanau Attention

### Multi-Head Attention

### Self-Attention dan Positional Encoding

### Transformer